# TP Estimation de Proportions avec Dataset Digits

Dans ce TP, nous allons explorer l'estimation par intervalle de confiance pour une proportion en utilisant le jeu de données `digits` de scikit-learn. Nous allons estimer la proportion de certains chiffres dans le dataset et construire des intervalles de confiance.

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

# Configuration de l'affichage
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(0)  # Pour la reproductibilité

## 1. Chargement et exploration des données

Nous commençons par charger le jeu de données `digits` qui contient des images 8x8 de chiffres manuscrits (de 0 à 9).

In [ ]:
# Chargement du dataset digits
digits = load_digits()
X = digits.data
y = digits.target

# Affichage de quelques informations sur le dataset
print(f"Forme des données: {X.shape}")
print(f"Nombre d'échantillons: {len(y)}")
print(f"Distribution des classes:")
for i in range(10):
    print(f"  Chiffre {i}: {np.sum(y == i)} échantillons")

Visualisons quelques exemples d'images de chiffres :

In [ ]:
# Affichage de quelques exemples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    # Sélection aléatoire d'un exemple pour chaque chiffre
    idx = np.random.choice(np.where(y == i)[0])
    ax.imshow(digits.images[idx], cmap='gray')
    ax.set_title(f'Chiffre: {y[idx]}')
    ax.axis('off')
plt.tight_layout()
plt.show()

## 2. Définition des événements binaires pour l'estimation de proportions

Pour estimer des proportions, nous devons définir des événements binaires (succès/échec). Nous allons considérer plusieurs cas :
1. La proportion de chiffres pairs
2. La proportion de chiffres supérieurs ou égaux à 5
3. La proportion d'un chiffre spécifique (ex: le chiffre 7)

In [ ]:
# Définition des événements binaires
est_pair = (y % 2 == 0).astype(int)  # 1 si le chiffre est pair, 0 sinon
est_grand = (y >= 5).astype(int)     # 1 si le chiffre est >= 5, 0 sinon
est_sept = (y == 7).astype(int)      # 1 si le chiffre est 7, 0 sinon

# Calcul des proportions réelles dans l'ensemble complet
p_pair_reel = np.mean(est_pair)
p_grand_reel = np.mean(est_grand)
p_sept_reel = np.mean(est_sept)

print(f"Proportion réelle de chiffres pairs: {p_pair_reel:.4f}")
print(f"Proportion réelle de chiffres >= 5: {p_grand_reel:.4f}")
print(f"Proportion réelle de chiffres = 7: {p_sept_reel:.4f}")

## 3. Estimation par intervalle de confiance pour une proportion

Nous allons maintenant diviser nos données en ensemble d'apprentissage et de test, puis estimer les proportions sur l'ensemble d'apprentissage et construire des intervalles de confiance.

In [ ]:
# Division en ensembles d'apprentissage et de test
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42)

# Calcul des événements binaires sur l'ensemble d'apprentissage
est_pair_train = (y_train % 2 == 0).astype(int)
est_grand_train = (y_train >= 5).astype(int)
est_sept_train = (y_train == 7).astype(int)

# Taille de l'échantillon
taille_echantillon = len(y_train)
print(f"Taille de l'échantillon d'apprentissage: {taille_echantillon}")

### 3.1 Intervalle de confiance avec l'approximation normale

Pour n assez grand (généralement n*p ≥ 5 et n*(1-p) ≥ 5), on peut utiliser l'approximation normale. L'erreur standard de $\hat{p}$ est donnée par $\sqrt{\frac{p(1-p)}{n}}$.

In [ ]:
# Fonction pour calculer l'intervalle de confiance avec l'approximation normale
def intervalle_confiance_normal(echantillon, alpha=0.05):
    p_hat = np.mean(echantillon)
    n = len(echantillon)
    z_alpha_2 = stats.norm.ppf(1 - alpha/2)
    erreur_standard = np.sqrt(p_hat * (1 - p_hat) / n)
    
    ic_inf = max(0, p_hat - z_alpha_2 * erreur_standard)
    ic_sup = min(1, p_hat + z_alpha_2 * erreur_standard)
    
    return p_hat, ic_inf, ic_sup

# Calcul des estimations et intervalles de confiance
p_pair_hat, ic_inf_pair, ic_sup_pair = intervalle_confiance_normal(est_pair_train)
p_grand_hat, ic_inf_grand, ic_sup_grand = intervalle_confiance_normal(est_grand_train)
p_sept_hat, ic_inf_sept, ic_sup_sept = intervalle_confiance_normal(est_sept_train)

# Affichage des résultats
print("Méthode de l'approximation normale:")
print(f"Proportion de chiffres pairs:")
print(f"  - Proportion réelle: {p_pair_reel:.4f}")
print(f"  - Proportion estimée: {p_pair_hat:.4f}")
print(f"  - Intervalle de confiance à 95%: [{ic_inf_pair:.4f}, {ic_sup_pair:.4f}]")
print(f"  - La proportion réelle est dans l'intervalle: {ic_inf_pair <= p_pair_reel <= ic_sup_pair}")

print(f"\nProportion de chiffres >= 5:")
print(f"  - Proportion réelle: {p_grand_reel:.4f}")
print(f"  - Proportion estimée: {p_grand_hat:.4f}")
print(f"  - Intervalle de confiance à 95%: [{ic_inf_grand:.4f}, {ic_sup_grand:.4f}]")
print(f"  - La proportion réelle est dans l'intervalle: {ic_inf_grand <= p_grand_reel <= ic_sup_grand}")

print(f"\nProportion de chiffres = 7:")
print(f"  - Proportion réelle: {p_sept_reel:.4f}")
print(f"  - Proportion estimée: {p_sept_hat:.4f}")
print(f"  - Intervalle de confiance à 95%: [{ic_inf_sept:.4f}, {ic_sup_sept:.4f}]")
print(f"  - La proportion réelle est dans l'intervalle: {ic_inf_sept <= p_sept_reel <= ic_sup_sept}")

Visualisons les intervalles de confiance pour les trois proportions :

In [ ]:
# Fonction pour visualiser l'intervalle de confiance
def visualiser_ic(p_hat, ic_inf, ic_sup, p_reel, titre):
    erreur_standard = (ic_sup - ic_inf) / (2 * stats.norm.ppf(0.975))
    
    plt.figure(figsize=(10, 6))
    
    # Densité de la loi normale de la proportion échantillonnale
    x = np.linspace(max(0, p_hat - 4*erreur_standard), min(1, p_hat + 4*erreur_standard), 1000)
    y = stats.norm.pdf(x, p_hat, erreur_standard)
    
    plt.plot(x, y, 'b-', linewidth=2, label='Distribution de la proportion échantillonnale')
    
    # Coloration de l'intervalle de confiance
    x_ic = np.linspace(max(0, ic_inf), min(1, ic_sup), 100)
    y_ic = stats.norm.pdf(x_ic, p_hat, erreur_standard)
    plt.fill_between(x_ic, y_ic, color='skyblue', alpha=0.4, label='Intervalle de confiance à 95%')
    
    # Ajout des lignes verticales
    plt.axvline(x=p_reel, color='r', linestyle='--', label='Proportion réelle')
    plt.axvline(x=p_hat, color='g', linestyle='-', label='Proportion estimée')
    plt.axvline(x=max(0, ic_inf), color='b', linestyle=':', label='Bornes de l\'intervalle')
    plt.axvline(x=min(1, ic_sup), color='b', linestyle=':')
    
    plt.title(titre)
    plt.xlabel('Valeur de la proportion')
    plt.ylabel('Densité')
    plt.legend()
    plt.grid(True)
    plt.show()

# Visualisation des intervalles de confiance
visualiser_ic(p_pair_hat, ic_inf_pair, ic_sup_pair, p_pair_reel, 'Intervalle de confiance pour la proportion de chiffres pairs')
visualiser_ic(p_grand_hat, ic_inf_grand, ic_sup_grand, p_grand_reel, 'Intervalle de confiance pour la proportion de chiffres >= 5')
visualiser_ic(p_sept_hat, ic_inf_sept, ic_sup_sept, p_sept_reel, 'Intervalle de confiance pour la proportion de chiffres = 7')

### 3.2 Intervalle de Wilson

L'intervalle de Wilson est une méthode plus précise pour estimer l'intervalle de confiance d'une proportion, notamment pour les petits échantillons ou les proportions proches de 0 ou 1.

In [ ]:
# Fonction pour calculer l'intervalle de Wilson
def intervalle_wilson(echantillon, alpha=0.05):
    p_hat = np.mean(echantillon)
    n = len(echantillon)
    X = int(p_hat * n)  # Nombre de succès
    z_alpha_2 = stats.norm.ppf(1 - alpha/2)
    
    # Terme central
    centre = (X + z_alpha_2**2/2) / (n + z_alpha_2**2)
    
    # Terme de marge
    marge = z_alpha_2 * np.sqrt((X * (n - X) / n + z_alpha_2**2/4) / (n + z_alpha_2**2))
    
    # Limites de l'intervalle
    ic_inf = max(0, centre - marge)
    ic_sup = min(1, centre + marge)
    
    return p_hat, ic_inf, ic_sup

# Calcul des estimations et intervalles de Wilson
p_pair_hat_w, ic_inf_pair_w, ic_sup_pair_w = intervalle_wilson(est_pair_train)
p_grand_hat_w, ic_inf_grand_w, ic_sup_grand_w = intervalle_wilson(est_grand_train)
p_sept_hat_w, ic_inf_sept_w, ic_sup_sept_w = intervalle_wilson(est_sept_train)

# Affichage des résultats
print("Méthode de Wilson:")
print(f"Proportion de chiffres pairs:")
print(f"  - Proportion réelle: {p_pair_reel:.4f}")
print(f"  - Proportion estimée: {p_pair_hat_w:.4f}")
print(f"  - Intervalle de Wilson à 95%: [{ic_inf_pair_w:.4f}, {ic_sup_pair_w:.4f}]")
print(f"  - La proportion réelle est dans l'intervalle: {ic_inf_pair_w <= p_pair_reel <= ic_sup_pair_w}")

print(f"\nProportion de chiffres >= 5:")
print(f"  - Proportion réelle: {p_grand_reel:.4f}")
print(f"  - Proportion estimée: {p_grand_hat_w:.4f}")
print(f"  - Intervalle de Wilson à 95%: [{ic_inf_grand_w:.4f}, {ic_sup_grand_w:.4f}]")
print(f"  - La proportion réelle est dans l'intervalle: {ic_inf_grand_w <= p_grand_reel <= ic_sup_grand_w}")

print(f"\nProportion de chiffres = 7:")
print(f"  - Proportion réelle: {p_sept_reel:.4f}")
print(f"  - Proportion estimée: {p_sept_hat_w:.4f}")
print(f"  - Intervalle de Wilson à 95%: [{ic_inf_sept_w:.4f}, {ic_sup_sept_w:.4f}]")
print(f"  - La proportion réelle est dans l'intervalle: {ic_inf_sept_w <= p_sept_reel <= ic_sup_sept_w}")

## 4. Comparaison des méthodes pour différentes tailles d'échantillon

Nous allons maintenant comparer les deux méthodes (normale et Wilson) pour différentes tailles d'échantillon et évaluer leur performance en termes de taux de couverture et de largeur d'intervalle.

In [ ]:
# Différentes tailles d'échantillon à tester
tailles = [20, 50, 100, 200, 500, 1000]
alpha = 0.05  # Niveau de confiance de 95%
n_simulations = 500  # Nombre de simulations pour chaque taille

# Nous allons nous concentrer sur la proportion de chiffres = 7 (cas d'une proportion faible)
p_reel = p_sept_reel
est_event = est_sept

# Préparation des structures pour stocker les résultats
couverture_norm = []
couverture_wilson = []
largeurs_norm = []
largeurs_wilson = []

for n in tailles:
    compteur_norm = 0
    compteur_wilson = 0
    largeur_norm_total = 0
    largeur_wilson_total = 0
    
    for _ in range(n_simulations):
        # Échantillonnage avec remplacement
        indices = np.random.choice(len(est_event), size=n, replace=True)
        echantillon = est_event[indices]
        
        # Méthode normale
        _, ic_inf_n, ic_sup_n = intervalle_confiance_normal(echantillon, alpha)
        
        # Méthode de Wilson
        _, ic_inf_w, ic_sup_w = intervalle_wilson(echantillon, alpha)
        
        # Vérification de la couverture
        if ic_inf_n <= p_reel <= ic_sup_n:
            compteur_norm += 1
        if ic_inf_w <= p_reel <= ic_sup_w:
            compteur_wilson += 1
            
        # Calcul des largeurs
        largeur_norm_total += (ic_sup_n - ic_inf_n)
        largeur_wilson_total += (ic_sup_w - ic_inf_w)
    
    # Moyenne des résultats
    couverture_norm.append(compteur_norm / n_simulations)
    couverture_wilson.append(compteur_wilson / n_simulations)
    largeurs_norm.append(largeur_norm_total / n_simulations)
    largeurs_wilson.append(largeur_wilson_total / n_simulations)

    print(f"Taille {n} terminée.")

In [ ]:
# Visualisation des résultats
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Taux de couverture
ax1.plot(tailles, couverture_norm, 'bo-', linewidth=2, label='Méthode normale')
ax1.plot(tailles, couverture_wilson, 'ro-', linewidth=2, label='Méthode de Wilson')
ax1.axhline(y=1-alpha, color='k', linestyle='--', label='Niveau nominal (95%)')
ax1.set_title('Taux de couverture des intervalles de confiance (Chiffre 7)')
ax1.set_xlabel('Taille de l\'échantillon')
ax1.set_ylabel('Taux de couverture')
ax1.set_xscale('log')
ax1.legend()
ax1.grid(True)

# Largeur des intervalles
ax2.plot(tailles, largeurs_norm, 'bo-', linewidth=2, label='Méthode normale')
ax2.plot(tailles, largeurs_wilson, 'ro-', linewidth=2, label='Méthode de Wilson')
ax2.set_title('Largeur moyenne des intervalles de confiance (Chiffre 7)')
ax2.set_xlabel('Taille de l\'échantillon')
ax2.set_ylabel('Largeur moyenne')
ax2.set_xscale('log')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

# Rendu de TP

Faire la meme analyse sur le dataset digits en considerant:
- La proportion du chiffre 1
- La proportion des chiffres entre 3 et 7 inclus